In [1]:
import nltk
import pandas as pd
import numpy as np
from processing import *
from sentiment import *

# PREPARASI *TRAINING DATASET*

In [7]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]


# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos.iloc[:int(len(all_pos)*0.8),:]
train_neg = all_neg.iloc[:int(len(all_neg)*0.8),:]
test_pos = all_pos.iloc[:int(len(all_pos)*0.2):]
test_neg = all_neg.iloc[:int(len(all_neg)*0.2):]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [11]:
ld = len(dataset)
trp = len(train_pos)
tng = len(train_neg)
tsp = len(test_pos)
tsn = len(test_neg)
print(f"dataset: {ld} \n train pos: {trp} \n train neg: {tng} \n test pos: {tsp} \n test neg: {tsn}")

dataset: 1300 
 train pos: 511 
 train neg: 528 
 test pos: 127 
 test neg: 132


# PELATIHAN *MODEL CLASSIFIER*

In [12]:
x = train_x['text']
y = train_y
kamus_frek = kamus_freq(x, y)

In [13]:
logprior, loglikelihood = train_nbc(kamus_frek, train_x, train_y)

## HASIL PENGUJIAN MODEL
- data *training*
- data *testing*

In [14]:
hasil_uji = test_nbc(x, y, logprior, loglikelihood)
hasil_uji

0.9749759384023099

In [33]:
test = test_nbc(test_x['text'], test_y, logprior, loglikelihood)
test

## UJI KESELURUHAN *DATASET*

In [ ]:
data_test = test_nbc(dataset['text'], dataset['label'], logprior, loglikelihood)
data_test

## *CONFUSION MATRIX*

In [ ]:
matrix = dataset
matrix

# UJI CLASSIFIER PADA ARTIKEL BERITA

In [23]:
berita = open('berita.txt', 'r')
berita = berita.read()

## SENTIMEN ARTIKEL KESELURUHAN

In [32]:
predict_nbc(berita, logprior, loglikelihood)

-33.3003778808566

## SENTIMEN ARTIKEL PER_KALIMAT

In [28]:
from nltk.tokenize import sent_tokenize
x = pd.DataFrame()
kalimat = sent_tokenize(berita)
x['kalimat'] = kalimat
x['kalimat_normal'] = x['kalimat'].apply(lambda x: normalisasi(x))

In [30]:
x['label_prediksi'] = x['kalimat'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [31]:
x

,kalimat,kalimat_normal,label_prediksi
0,"BEIJING, KOMPAS.com - China pada Selasa (10/8/...","[beijing, kompas, china, selasa, tari, utus, l...",-2.345609
1,Taiwan pada Juli mengatakan akan mendirikan ka...,"[taiwan, juli, kantor, wakil, vilnius, kota, l...",-2.631575
2,China menganggap Taiwan bukanlah negara yang b...,"[china, anggap, taiwan, negara, daulat, wilayah]",-2.521066
3,Kementerian Luar Negeri China marah dan pada S...,"[menteri, negeri, china, marah, selasa, perint...",1.176804
4,Keputusan Lithuania mengizinkan Taiwan mendiri...,"[putus, lithuania, izin, taiwan, kantor, wakil...",-2.183572
5,"Selain itu, Kementerian Luar Negeri China meni...","[menteri, negeri, china, nilai, lithuania, ber...",-3.240881
6,China yang mengklaim Taiwan sebagai bagiannya ...,"[china, klaim, taiwan, janji, rebut, keras]",-1.711852
7,China mencoba membuat Taiwan tetap terisolasi ...,"[china, coba, taiwan, isolasi, panggung, dunia...",-1.746056
8,Kementerian Luar Negeri China pada Selasa (10/...,"[menteri, negeri, china, selasa, desak, lithua...",0.664262
9,Kementerian Luar Negeri Lithuania menanggapi d...,"[menteri, negeri, lithuania, tanggap, kecewa, ...",-1.541953


## SENTIMEN ARTIKEL PER_KATA

In [18]:
# * predict_nbc per-kata
def predict_perkata(text, logpri, loglik):
    l = []
    for w in text.split():
        x = predict_nbc(w, logpri, loglik)
        l.append({
            w,
            x
            })
    return l

predict_perkata(berita, logprior, loglikelihood)

[{-0.03272669350255697, 'BEIJING,'},
 {-0.03272669350255697, 'KOMPAS.com'},
 {'-', -0.03272669350255697},
 {-0.03272669350255697, 'China'},
 {-0.03272669350255697, 'pada'},
 {-0.03272669350255697, 'Selasa'},
 {'(10/8/2021)', -0.03272669350255697},
 {-0.03272669350255697, 'meminta'},
 {0.6587042394965325, 'penarikan'},
 {-0.03272669350255697, 'utusan'},
 {-0.03272669350255697, 'Lithuania'},
 {-0.03272669350255697, 'untuk'},
 {-0.03272669350255697, 'Beijing,'},
 {-0.03272669350255697, 'setelah'},
 {-0.03272669350255697, 'Vilnius'},
 {-0.03272669350255697, 'mengizinkan'},
 {-0.03272669350255697, 'Taiwan'},
 {-0.03272669350255697, 'mendirikan'},
 {-0.03272669350255697, 'kantor'},
 {-0.03272669350255697, 'di'},
 {-1.826202410291468, 'negaranya,'},
 {-0.5452685648294038, 'sebuah'},
 {0.6587042394965325, 'langkah'},
 {-0.03272669350255697, 'yang'},
 {-0.03272669350255697, 'dianggap'},
 {-0.7275901216233582, 'provokatif'},
 {-0.03272669350255697, 'oleh'},
 {-0.7275901216233582, 'pemerintah'},
